In [1]:
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from narwhals.selectors import categorical

In [2]:
PROCESSED_DATA = Path("../data/processed/")
FIGURES_DATA = Path("../reports/figures/")
INTERIM_DATA = Path("../data/interim/")

In [3]:
df = pd.read_parquet(INTERIM_DATA / "merge_data.parquet")

In [4]:
df.columns

Index(['transaction_id', 'timestamp', 'user_id', 'merchant_id', 'amount',
       'channel', 'currency', 'device', 'location', 'payment_method',
       'is_international', 'session_length_seconds', 'is_first_time_merchant',
       'is_fraud', 'age', 'sex', 'education', 'primary_source_of_income',
       'sum_of_monthly_installments', 'sum_of_monthly_expenses',
       'country_user', 'signup_date', 'risk_score', 'category',
       'country_merchant', 'trust_score', 'number_of_alerts_last_6_months',
       'avg_transaction_amount', 'account_age_months', 'has_fraud_history'],
      dtype='object')

In [5]:
df.drop(columns=["currency", "user_id", "merchant_id", "location"], inplace=True)

In [6]:
df.columns

Index(['transaction_id', 'timestamp', 'amount', 'channel', 'device',
       'payment_method', 'is_international', 'session_length_seconds',
       'is_first_time_merchant', 'is_fraud', 'age', 'sex', 'education',
       'primary_source_of_income', 'sum_of_monthly_installments',
       'sum_of_monthly_expenses', 'country_user', 'signup_date', 'risk_score',
       'category', 'country_merchant', 'trust_score',
       'number_of_alerts_last_6_months', 'avg_transaction_amount',
       'account_age_months', 'has_fraud_history'],
      dtype='object')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   transaction_id                  500000 non-null  object        
 1   timestamp                       500000 non-null  datetime64[ns]
 2   amount                          500000 non-null  float64       
 3   channel                         500000 non-null  object        
 4   device                          500000 non-null  object        
 5   payment_method                  500000 non-null  object        
 6   is_international                500000 non-null  int64         
 7   session_length_seconds          500000 non-null  int64         
 8   is_first_time_merchant          500000 non-null  int64         
 9   is_fraud                        500000 non-null  int64         
 10  age                             500000 non-null  int64  

In [9]:
df['education'] = df['education'].fillna('Missing')

In [10]:
df.drop(columns=["transaction_id"], inplace=True)

In [13]:
pd.get_dummies(df['education'], dtype=int)

,Bachelor,High School,Master,Missing,PhD
0,1,0,0,0,0
1,0,0,0,0,1
2,1,0,0,0,0
3,0,0,1,0,0
4,0,0,0,0,1
...,...,...,...,...,...
499995,1,0,0,0,0
499996,0,1,0,0,0
499997,1,0,0,0,0
499998,0,1,0,0,0


In [16]:
categorical_cols = ["channel", "device", "payment_method", "sex", "education", "primary_source_of_income", "category"]

In [28]:
df_encoded = pd.get_dummies(df, columns=categorical_cols, dtype=int, drop_first=True)

In [29]:
df_encoded.shape

(500000, 42)

In [31]:
df.columns

Index(['timestamp', 'amount', 'channel', 'device', 'payment_method',
       'is_international', 'session_length_seconds', 'is_first_time_merchant',
       'is_fraud', 'age', 'sex', 'education', 'primary_source_of_income',
       'sum_of_monthly_installments', 'sum_of_monthly_expenses',
       'country_user', 'signup_date', 'risk_score', 'category',
       'country_merchant', 'trust_score', 'number_of_alerts_last_6_months',
       'avg_transaction_amount', 'account_age_months', 'has_fraud_history'],
      dtype='object')

In [30]:
from sklearn.preprocessing import StandardScaler

In [32]:
numerical_cols = ["amount", "session_length_seconds", "age", "sum_of_monthly_installments", "risk_score", "trust_score",
				  "number_of_alerts_last_6_months", "avg_transaction_amount", "account_age_months"]

In [33]:
scaler = StandardScaler()

In [35]:
df_encoded[numerical_cols] = scaler.fit_transform(df_encoded[numerical_cols])

In [37]:
df_encoded.describe()

,timestamp,amount,is_international,session_length_seconds,is_first_time_merchant,is_fraud,age,sum_of_monthly_installments,sum_of_monthly_expenses,signup_date,...,primary_source_of_income_Retirement,primary_source_of_income_Savings,primary_source_of_income_Student Aid,primary_source_of_income_Unemployment,category_education,category_electronics,category_gaming,category_grocery,category_restaurants,category_travel
count,500000,5.000000e+05,500000.000000,5.000000e+05,500000.000000,500000.000000,5.000000e+05,5.000000e+05,500000.000000,500000,...,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000
mean,2022-12-31 16:45:04.497720576,8.754597e-17,0.928662,1.131397e-16,0.501248,0.084822,-1.078746e-16,-3.996092e-17,1182.346649,2022-10-06 13:27:43.660799744,...,0.165126,0.164698,0.162216,0.173432,0.140724,0.153108,0.127132,0.156492,0.134314,0.136254
min,2022-01-01 00:06:00,-9.985008e-01,0.000000,-1.730265e+00,0.000000,0.000000,-1.691432e+00,-9.928511e-01,0.040000,2020-04-21 00:00:00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2022-07-01 11:03:45,-7.110468e-01,1.000000,-8.636345e-01,0.000000,0.000000,-8.442505e-01,-7.048258e-01,353.660000,2021-07-09 00:00:00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2022-12-31 20:08:30,-3.071710e-01,1.000000,-2.919869e-03,1.000000,0.000000,2.930763e-03,-3.097876e-01,825.190000,2022-10-08 00:00:00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2023-07-02 08:45:15,3.852091e-01,1.000000,8.637103e-01,1.000000,0.000000,8.501120e-01,3.843131e-01,1650.440000,2023-12-29 00:00:00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2023-12-31 23:57:00,1.330139e+01,1.000000,1.730340e+00,1.000000,1.000000,1.697293e+00,9.437243e+00,10994.330000,2025-03-22 00:00:00,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
std,NaN,1.000001e+00,0.257389,1.000001e+00,0.499999,0.278617,1.000001e+00,1.000001e+00,1168.967960,NaN,...,0.371295,0.370908,0.368649,0.378621,0.347737,0.360092,0.333121,0.363322,0.340990,0.343058


In [38]:
df_encoded.to_parquet(INTERIM_DATA / "encoded_data.parquet")

In [39]:
df_encoded.shape

(500000, 42)